<a href="https://colab.research.google.com/github/MingmingFortez/AI_research/blob/main/Clothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U datasets transformers peft accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
import torch

In [3]:
# Load the fashion product dataset
dataset = load_dataset("agestau/preproc-fashion-products")

# Preview an example
dataset["train"][0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/454 [00:00<?, ?B/s]

(…)-00000-of-00001-a3cd901900de052e.parquet:   0%|          | 0.00/224M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/36145 [00:00<?, ? examples/s]

{'subCategory': 'Topwear',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80>,
 'text': 'Men Navy Blue Shirt'}

In [4]:
# Get the unique subcategory labels and create mappings
labels = list(set(example["subCategory"] for example in dataset["train"]))
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

# Encode labels
def encode(example):
    example["label"] = label2id[example["subCategory"]]
    return example

encoded_dataset = dataset.map(encode)


Map:   0%|          | 0/36145 [00:00<?, ? examples/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_dataset = encoded_dataset.map(tokenize, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/36145 [00:00<?, ? examples/s]

In [6]:
# Load base model for classification
base_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(labels)
)

# Configure and apply LoRA
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"]
)

model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 748,814 || all params: 67,713,052 || trainable%: 1.1059


In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./fashion-results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=10,
    report_to="none"  # disables wandb
)



In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer
)

trainer.train()


/tmp/ipython-input-10-1471395078.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,0.112100
20,0.089500
30,0.052800
40,0.123100
50,0.103300
60,0.119400
70,0.079400
80,0.095900
90,0.088200
100,0.083000


TrainOutput(global_step=6780, training_loss=0.02145805721686999, metrics={'train_runtime': 3905.9749, 'train_samples_per_second': 27.761, 'train_steps_per_second': 1.736, 'total_flos': 1.46166155025408e+16, 'train_loss': 0.02145805721686999, 'epoch': 3.0})

In [9]:
print(f"Number of classes: {len(label2id)}")
print(label2id)


Number of classes: 14
{'Socks': 0, 'Ties': 1, 'Watches': 2, 'Topwear': 3, 'Bottomwear': 4, 'Wallets': 5, 'Headwear': 6, 'Shoes': 7, 'Scarves': 8, 'Eyewear': 9, 'Jewellery': 10, 'Bags': 11, 'Dress': 12, 'Belts': 13}


In [15]:
from transformers import AutoModelForSequenceClassification, pipeline

# ID to label mapping (same as fine-tuned)
id2label = {
    0: "Socks", 1: "Ties", 2: "Watches", 3: "Topwear", 4: "Bottomwear", 5: "Wallets",
    6: "Headwear", 7: "Shoes", 8: "Scarves", 9: "Eyewear", 10: "Jewellery",
    11: "Bags", 12: "Dress", 13: "Belts"
}
model.config.id2label = {
    0: "Socks", 1: "Ties", 2: "Watches", 3: "Topwear", 4: "Bottomwear", 5: "Wallets",
    6: "Headwear", 7: "Shoes", 8: "Scarves", 9: "Eyewear", 10: "Jewellery",
    11: "Bags", 12: "Dress", 13: "Belts"
}
model.config.label2id = {v: k for k, v in model.config.id2label.items()}


# Base DistilBERT model (not fine-tuned)
base_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(id2label)
)
base_model.config.id2label = id2label
base_model.config.label2id = {v: k for k, v in id2label.items()}

# Pipelines for both
fine_tuned_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
base_pipe = pipeline("text-classification", model=base_model, tokenizer=tokenizer)

# Test input
examples = [
    "Red leather wallet with zipper",
    "Men's running shoes with extra grip",
    "Golden watch with stainless steel strap",
    "Wool winter scarf for women",
    "Black crew socks, cotton blend",
    "Elegant evening dress with lace"
]

# Compare predictions
for text in examples:
    base_output = base_pipe(text)[0]
    fine_output = fine_tuned_pipe(text)[0]

    base_label = base_output["label"]
    fine_label = fine_output["label"]

    print(f"\nReview: {text}")
    print(f"Base Model:      {base_label} ({round(base_output['score']*100, 2)}%)")
    print(f"Fine-Tuned Model: {fine_label} ({round(fine_output['score']*100, 2)}%)")



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Device set to use cuda:0



Review: Red leather wallet with zipper
Base Model:      Eyewear (8.02%)
Fine-Tuned Model: Wallets (82.19%)

Review: Men's running shoes with extra grip
Base Model:      Eyewear (8.39%)
Fine-Tuned Model: Shoes (99.99%)

Review: Golden watch with stainless steel strap
Base Model:      Eyewear (8.37%)
Fine-Tuned Model: Watches (99.63%)

Review: Wool winter scarf for women
Base Model:      Eyewear (8.37%)
Fine-Tuned Model: Scarves (99.72%)

Review: Black crew socks, cotton blend
Base Model:      Eyewear (8.23%)
Fine-Tuned Model: Socks (99.99%)

Review: Elegant evening dress with lace
Base Model:      Eyewear (8.23%)
Fine-Tuned Model: Dress (99.93%)


In [17]:
from transformers import pipeline, AutoModelForSequenceClassification

# Define your label mappings
id2label = {
    0: "Socks", 1: "Ties", 2: "Watches", 3: "Topwear", 4: "Bottomwear", 5: "Wallets",
    6: "Headwear", 7: "Shoes", 8: "Scarves", 9: "Eyewear", 10: "Jewellery",
    11: "Bags", 12: "Dress", 13: "Belts"
}
label2id = {v: k for k, v in id2label.items()}

# Assign label mappings to both models
base_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=14)
base_model.config.id2label = id2label
base_model.config.label2id = label2id

model.config.id2label = id2label
model.config.label2id = label2id

# Build pipelines
base_pipe = pipeline("text-classification", model=base_model, tokenizer=tokenizer)
fine_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Input loop
while True:
    user_input = input("\nEnter a fashion product description (or type 'exit' to quit): ")
    if user_input.lower() == "exit":
        break

    base_result = base_pipe(user_input)[0]
    fine_result = fine_pipe(user_input)[0]

    print(f"\n Base Model Prediction:     {base_result['label']} ({round(base_result['score'] * 100, 2)}%)")
    print(f" Fine-Tuned Model Prediction: {fine_result['label']} ({round(fine_result['score'] * 100, 2)}%)")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Device set to use cuda:0



Enter a fashion product description (or type 'exit' to quit): camisole

 Base Model Prediction:     Dress (8.6%)
 Fine-Tuned Model Prediction: Bottomwear (95.54%)

Enter a fashion product description (or type 'exit' to quit): shorts

 Base Model Prediction:     Dress (8.88%)
 Fine-Tuned Model Prediction: Bottomwear (99.99%)

Enter a fashion product description (or type 'exit' to quit): tshirt

 Base Model Prediction:     Dress (8.55%)
 Fine-Tuned Model Prediction: Topwear (99.97%)

Enter a fashion product description (or type 'exit' to quit): gown

 Base Model Prediction:     Dress (8.96%)
 Fine-Tuned Model Prediction: Dress (99.21%)

Enter a fashion product description (or type 'exit' to quit): sunglasses

 Base Model Prediction:     Dress (8.87%)
 Fine-Tuned Model Prediction: Eyewear (99.99%)

Enter a fashion product description (or type 'exit' to quit): exit
